In [1]:
import os
import sqlite3
import pandas as pd
import tqdm

In [2]:
db_name = '[Full_TFIDF]Credibility_research_20180906.db' #DB 파일명
# db 생성
con = sqlite3.connect( db_name )
cur = con.cursor()

In [469]:
sql = '''
select p.Post_id,p.Category,p.Title,u.Credibility
    FROM Post as p
    Left JOIN user as u
        ON p.User_id = u.User_id
    '''
cur.execute(sql)
rows = cur.fetchall()

In [470]:
col_name = [i[0] for i in cur.description]

In [471]:
test_df = pd.DataFrame(rows)

In [472]:
test_df.columns = col_name

In [473]:
test_df.columns.tolist()

['Post_id', 'Category', 'Title', 'Credibility']

In [474]:
test_df = test_df[test_df['Category'] != "무작위"].iloc[:16304]

In [477]:
test_df = test_df[test_df['Credibility']==1]

# 29_features

In [478]:
df_29 = pd.read_csv('[Final]Total_csv.csv',sep='\t')

In [479]:
df_29 = df_29[['Post_id','Question_count','First_ratio','Second_ratio','Tag_count','pos_ratio',
 'neg_ratio','subjectivity','polarity','senti_diffs_per_ref','Sticker_count','Text_len','Count_space_mistake',
   'Left','Center','Right','Justify',
   'img img img img img','img img img img text','img img img text img','img img text img img','img img text img text',
 'img text img img img','img text img img text','img text img text img','text img img img img','text img img img text',
 'text img img text img','text img text img img','text img text img text']]

# Text_29 features merge

In [480]:
test_df['Post_id'] = test_df['Post_id'].apply(lambda x : int(x))

/Users/yonggeol/miniconda3/envs/py/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [481]:
total_df = pd.merge(test_df,df_29)

In [482]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(total_df['Category'])
le.classes_
total_df['Category'] =le.transform(total_df['Category'])
total_df = total_df.dropna()

In [483]:
le.classes_

array(['IT·컴퓨터', '건강·의학', '공연·전시', '교육·학문', '국내여행', '드라마·방송', '등산·낚시·레저',
       '만화·애니', '맛집', '사진', '스포츠', '시사·인문·경제', '어학·외국어', '와인·술', '육아·결혼',
       '자동차', '차·커피·디저트', '패션·뷰티'], dtype=object)

In [484]:
from sklearn.utils import shuffle
total_df = shuffle(total_df)

In [485]:
text = total_df['Title']
y = total_df['Category']
others = total_df.drop(columns=['Title','Category'])

In [516]:
from sklearn.feature_extraction.text import TfidfVectorizer
#vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
vectorizer = TfidfVectorizer(max_df=0.01)
tfidf = vectorizer.fit_transform(text)

In [517]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(others) 
others_scaled = pd.DataFrame(scaler.transform(others),columns = others.columns)

In [518]:
tfidf_result = pd.DataFrame(tfidf.toarray())

In [519]:
x = pd.concat([tfidf_result,others],axis=1)

In [520]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , y , test_size=0.33, random_state=42)

In [521]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((6649, 30328), (6649,), (3275, 30328), (3275,))

In [522]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=2.0,random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
logreg.fit(x_train, y_train)

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=True)

In [523]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

0.5233587786259541

In [524]:
from sklearn.metrics import precision_score
precision_score(logreg.predict(x_test), y_test,average=None)

array([0.38922156, 0.315     , 0.49112426, 0.38285714, 0.20858896,
       0.14772727, 0.26315789, 0.6713615 , 0.63421829, 0.93647913,
       0.58088235, 0.25294118, 0.34      , 0.10526316, 0.04878049,
       0.70833333, 0.19191919, 0.464     ])

In [525]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logreg.predict(x_test)))

             precision    recall  f1-score   support

          0       0.89      0.39      0.54       167
          1       0.93      0.32      0.47       200
          2       0.94      0.49      0.65       169
          3       0.97      0.38      0.55       175
          4       0.71      0.21      0.32       163
          5       1.00      0.15      0.26        88
          6       0.87      0.26      0.40        76
          7       0.95      0.67      0.79       213
          8       0.72      0.63      0.68       339
          9       0.27      0.94      0.42       551
         10       0.88      0.58      0.70       272
         11       0.81      0.25      0.39       170
         12       1.00      0.34      0.51        50
         13       1.00      0.11      0.19        19
         14       1.00      0.05      0.09        82
         15       0.94      0.71      0.81       192
         16       0.95      0.19      0.32        99
         17       0.86      0.46      0.60   

# 실제 맛집 Test

In [459]:
class textclass:
    def Extract_structure_and_tag(User_id, Post_id):
        url = "http://blog.naver.com/PostView.nhn?blogId=" + User_id + "&logNo=" + Post_id + "&redirect=Dlog&widgetTypeCall=true"
        r = requests.get(url)
        bs = BeautifulSoup(re.sub('&nbsp;', ' ', r.text).encode("utf-8"), "html.parser")
        # structure
        structure = bs.find("div", {"id": "postViewArea"})
        if structure == None:
            structure = bs.find("div", {"class", "se_component_wrap sect_dsc __se_component_area"})

        structure_p_img_tag = structure.find_all(['p', 'img'])
        structure_dict = {'structure': structure, 'structure_p_img_tag': structure_p_img_tag}
        # structure_p_img_tag : p,img tag만 extract
        # structure : 모든 tag 가져오기
        return structure_dict

    # Extract_structure_and_tag 함수의 'structure_p_img_tag' 값을 가져와야함.

    def HTML_preprocessing(structure_p_img_tag):
        # only tag & text extract
        tag_list = []
        text_list = []
        for i in structure_p_img_tag:
            # p_tag만 불러오기
            if "<p" in (str(i)):
                tag_list.append('<p>')
                # img만 있을 때

                if '<img' in str(i):
                    for j in i:
                        try:
                            if len(j.text) > 1:
                                tag_list.append('<br>')
                                text_list.append(j.text)
                        except:
                            pass

                # img가 아닌 경우 span tag가 더 있을 때
                elif '<span' in str(i):
                    for j in i:
                        if '<br' in str(j):
                            text_list.append(j.text)
                            # br_tag가 2개 이상 있을 때

                            if len(j.findAll('br')) > 2:
                                for _ in range(0, len(j.findAll('br'))):
                                    tag_list.append('<br>')

                            # br_tag가 1개 있을 때
                            else:
                                tag_list.append('<br>')

                        # span은 있지만 br tag가 없을 때
                        else:
                            try:
                                text_list.append(j.text)
                            except:
                                pass

                # 그냥 p_tag만 있을 때 br_tag 추가
                else:
                    # 글이 있을 때
                    if len(i.text) > 1:
                        text_list.append(i.text)

                    # 글 없이 br tag만 있을 때
                    else:
                        tag_list.append('<br>')
                        text_list.append(i.text)

                # P_tag 끝맽음
                tag_list.append('</p>')

            else:
                tag_list.append('<img>')

        text_list = list(map(remove_odd, text_list))
        filter_text = list(filter(lambda x: len(x) > 1, text_list))

        Text = " ".join(list(filter(lambda x: len(x) > 1, map(lambda x: x.strip(), text_list))))
        Text = re.sub('\n', '', Text)
        Text = re.sub('\t', '', Text)
        Space_text = " ".join(Spacing_text(filter_text))
        Count_space_mistake = len(Space_text) - len(Text)

        # only tag
        Structure_only_tag = "|".join(tag_list)
        Structure_only_tag_df = pd.DataFrame({'text': [Structure_only_tag]})
        array_temp = Structure_only_tag_df['text'].apply(
            lambda x: " img ".join(list(map(lambda x: 'text' if len(x) > 3 else '', x.split('<img>')))).strip().replace(
                '  ', ' ')).values
        refined_structure = ''.join(array_temp)

        HTML_preprocessing = {'Text': Text, 'refined_structure': refined_structure,
                              'Count_space_mistake': Count_space_mistake}

        return HTML_preprocessing

    def sentimental_analysis(text):
        pos_word_list = []
        neg_word_list = []
        
        pos_ratio = 0.000000001
        neg_ratio = 0.000000001
        subjectivity = 0.000000001
        polarity = 0.000000001
        senti_diffs_per_ref = 0.000000001

        if text == '':
            sentiment_dict = {'pos_ratio': pos_ratio, 'neg_ratio': neg_ratio, 'subjectivity': subjectivity,
                              'polarity': polarity, 'senti_diffs_per_ref': senti_diffs_per_ref}
            return sentiment_dict, pos_word_list, neg_word_list
        else:
            pos = 0
            neg = 0
            text = text.split(' ')
            n = len(text)
            for i in text:
                i = remove_odd(i)
                pre = kkma.pos(i)
                test = ';'.join(['/'.join(i) for i in pre])
                if test in word_list:
                    if label[word_list.index(test)] == 'POS':
                        pos += 1
                        pos_word_list.append(test)
                    elif label[word_list.index(test)] == 'NEG':
                        neg += 1
                        neg_word_list.append(test)
            try:
                pos_ratio = pos / n
            except:
                pass
            try:
                neg_ratio = neg / n
            except:
                pass
            try:
                subjectivity = (neg + pos) / n
            except:
                pass
            try:
                polarity = (neg - pos) / (neg + pos)
            except:
                pass
            try:
                senti_diffs_per_ref = (pos - neg) / n
            except:
                pass

            sentiment_dict = {'pos_ratio': pos_ratio, 'neg_ratio': neg_ratio, 'subjectivity': subjectivity,
                              'polarity': polarity, 'senti_diffs_per_ref': senti_diffs_per_ref}
            return sentiment_dict, pos_word_list, neg_word_list

    def check_First_second(Text):
        first_person = ['나/NP', '저/NP', '내/NP', '제/NP', '저희/NP', '우리/NP']
        second_person = ['너/NP', '자네/NP', '당신/NP', '그대/NP', '그쪽/NP', '너희/NP', '자기/NP']
        First = 0
        Second = 0
        if Text == '':
            check_First_second_dict = {'First': First, 'Second': Second}
            return check_First_second_dict
        else:
            text = kkma.pos(Text)
            for i in text:
                temp = "/".join(i)
                if temp in first_person:
                    First += len(temp.split('/')[0])
                if temp in second_person:
                    Second += len(temp.split('/')[0])
            check_First_second_dict = {'First_ratio': First/len(Text), 'Second_ratio': Second/len(Text)}
            return check_First_second_dict

In [460]:
def Spacing_text(text_list):
    spacing_list = []
    for i in text_list:
        if len(i) < 197:
            spacing_list.append(spacing(i))
        else:
            iteration = int(len(i) / 197)
            mod = len(i) % 197
            start = 0
            end = 197
            check = 0
            while True:
                # 시행횟수 < 몫
                if check < iteration:
                    spacing_list.append(spacing(i[start:end]))
                    start += 197
                    end += 197
                    check += 1
                else:
                    # 마지막 횟수 + 나머지 더 slice
                    spacing_list.append(spacing(i[iteration * 197:(iteration * 197) + mod]))
                    break
    return spacing_list

def remove_odd(x):
    x = re.sub("nbsp", " ", x)
    x = re.sub("\xa0", "", x)
    x = re.sub("\u200b", "", x)
    x = re.sub("\n", "", x)
    x = re.sub("\t", "", x)
    x = re.sub('   ', ' ', x)
    return x

def tfidf_vectorizer(Text):
    try:
        return v_load.transform([Text]).toarray().flatten()
    except:
        return ''

In [461]:
# ex1 https://blog.naver.com/you-n-mi?Redirect=Log&logNo=221352751222
# ex2 https://blog.naver.com/sky_sea11?Redirect=Log&logNo=221216249472
# ex3 https://blog.naver.com/jhforever48/221154182850
# ex4 https://blog.naver.com/soundbross?Redirect=Log&logNo=221403690848

In [462]:
User_id = ['you-n-mi','sky_sea11','jhforever48','soundbross','senti54','0105114a']
Post_id = ['221352751222','221216249472','221154182850','221403690848','221401308842','221269157353']

In [463]:
from bs4 import BeautifulSoup 
import re
import requests
from selenium import webdriver
from pykospacing import spacing
import pandas as pd
from konlpy.tag import Kkma
import numpy as np
import pickle

In [464]:
text_list = []
for User_id,Post_id in list(zip(User_id, Post_id)):
    url = "http://blog.naver.com/PostView.nhn?blogId=" + User_id + "&logNo=" + Post_id + "&redirect=Dlog&widgetTypeCall=true"
    mobile_url = "http://m.blog.naver.com/PostView.nhn?blogId="+ User_id
    opening_url = 'http://blog.naver.com/profile/intro.nhn?blogId='+ User_id
    structure = textclass.Extract_structure_and_tag(User_id,Post_id)
    all_tag = structure['structure']
    p_img_tag = structure['structure_p_img_tag']
    HTML_preprocessing = textclass.HTML_preprocessing(p_img_tag)
    text = HTML_preprocessing['Text']
    text_list.append(text)

In [526]:
url = "http://blog.naver.com/PostView.nhn?blogId=" + User_id + "&logNo=" + Post_id + "&redirect=Dlog&widgetTypeCall=true"
r = requests.get(url)
bs = BeautifulSoup(re.sub('&nbsp;', ' ', r.text).encode("utf-8"), "html.parser")

In [527]:
structure = bs.find("div", {"id": "postViewArea"})

In [532]:
bs.find("div", {"class": "wrap_tag"})

<div class="wrap_tag">
<div class="_param(false|false|3)" id="tagList_221269157353" style="display:none;cursor:default;">
<strong class="blind">태그</strong>
</div>
<div style="display:none">
<span class="wrap_input_text">
<input class="input_text pcol2 _tagInputArea" title="태그를 입력하세요." type="text" value="">
<i class="aline"></i><i class="pcol2b"></i>
</input></span>
<span class="wrap_btn"><a class="btn pcol2 _cancelTag _returnFalse _param(221269157353)" href="#">취소<i class="aline"></i></a>
<a class="btn pcol2 _saveTag _rosRestrict _returnFalse _param(221269157353)" href="#">확인<i class="aline"></i></a></span>
</div>
</div>